# Agent IA Notion


###  Importation des packages

In [9]:
from agents.mcp.server import MCPServerStdio, MCPServer
from agents import Agent, Runner, gen_trace_id, trace, ModelSettings
from openai.types.responses import ResponseTextDeltaEvent
import os
import asyncio

from dotenv import load_dotenv
load_dotenv()

True

In [10]:
import logging
# Suppress httpx INFO logs to reduce console output
logging.getLogger("httpx").setLevel(logging.WARNING)

##  Comment obtenir une clé API Notion (token d'intégration interne)

###  Étape 1 : Créer une intégration

1. Aller sur [https://www.notion.com/my-integrations](https://www.notion.com/my-integrations)
2. Cliquer sur **"New integration"**
3. Remplir :

   * Le **nom** de l’intégration
   * Sélectionner l’espace de travail
   * (Facultatif) Ajouter un logo
4. Cliquer sur **"Submit"**



### Étape 2 : Récupérer la clé API

* Une fois l’intégration créée, vous verrez un champ **"Internal Integration Token"**.
* **Copiez ce jeton** : c’est votre **clé API Notion**
*  Gardez-le **secret** et stockez-le dans un fichier `.env` si possible.


### Étape 3 : Donner l’accès à une page ou base de données

1. Ouvrir la page ou la base Notion
2. Cliquer sur **"Partager"** > **"Ajouter une connexion"**
3. Sélectionner l’intégration que vous avez créée
4. Valider pour lui donner les droits d’accès


###  Étape 4 : Utiliser la clé  avec une variable d’environnement


Fichier `.env` :

```env
NOTION_API_KEY=your_token_here
```
###  Bonnes pratiques

* Ne publiez jamais votre clé dans un dépôt public
* Donnez des accès **seulement aux pages nécessaires**
* Vérifiez que la version utilisée est compatible : ajoutez toujours l'en-tête `Notion-Version: 2022-06-28` si vous utilisez des requêtes manuelles




In [11]:
# import notion api key
from dotenv import load_dotenv                              # Importe la fonction pour charger les variables d’environnement depuis un fichier .env
import os                                                   # Permet d’accéder aux variables d’environnement via os.getenv()

load_dotenv(dotenv_path=".env", override=True)              # Charge explicitement le fichier .env et écrase les variables existantes si nécessaire

notion_key = os.getenv("NOTION_API_KEY")                       # Récupère la clé API OpenAI depuis l’environnement
print("Clé API Notion chargée :", notion_key[:8] + "..." if notion_key else "Aucune clé détectée")  # Affiche un extrait de la clé ou un message d’erreur

api_key = os.getenv("OPENAI_API_KEY")                       # Récupère la clé API OpenAI depuis l’environnement
print("Clé API GPT chargée :", api_key[:8] + "..." if api_key else "Aucune clé détectée")  # Affiche un extrait de la clé ou un message d’erreur




Clé API Notion chargée : ntn_4790...
Clé API GPT chargée : sk-proj-...


### explore MCP server tools

In [12]:
async with MCPServerStdio(
    params={
        "command": "npx",
        "args": ["-y", "@notionhq/notion-mcp-server"],
        "env": {
        "OPENAPI_MCP_HEADERS": "{\"Authorization\": \"Bearer " + notion_key + "\", \"Notion-Version\": \"2022-06-28\" }"
        }
    }
) as server:
    tool_list = await server.list_tools()


In [13]:
toolname_list = [tool.name for tool in tool_list]
print(toolname_list)

['API-get-user', 'API-get-users', 'API-get-self', 'API-post-database-query', 'API-post-search', 'API-get-block-children', 'API-patch-block-children', 'API-retrieve-a-block', 'API-update-a-block', 'API-delete-a-block', 'API-retrieve-a-page', 'API-patch-page', 'API-post-page', 'API-create-a-database', 'API-update-a-database', 'API-retrieve-a-database', 'API-retrieve-a-page-property', 'API-retrieve-a-comment', 'API-create-a-comment']


In [14]:
[tool.description for tool in tool_list]

['Retrieve a user\nError Responses:\n400: 400',
 'List all users\nError Responses:\n400: 400',
 "Retrieve your token's bot user",
 'Query a database',
 'Search by title',
 'Retrieve block children',
 'Append block children',
 'Retrieve a block',
 'Update a block',
 'Delete a block',
 'Retrieve a page',
 'Update page properties',
 'Create a page',
 'Create a database',
 'Update a database',
 'Retrieve a database',
 'Retrieve a page property item',
 'Retrieve comments',
 'Create comment']

In [15]:
!pip install nest_asyncio


### Créer le point d'entrée principal avec le serveur MCP


In [16]:
import os
import nest_asyncio
import asyncio
from dotenv import load_dotenv
import traceback

# Pour autoriser asyncio dans Jupyter
nest_asyncio.apply()



# Fonction principale
async def main():
    from agents.mcp.server import MCPServerStdio  # importer ici pour éviter certains conflits éventuels

    try:
        print(" Clé API détectée :", notion_key[:10] + "...")
        async with MCPServerStdio(
            params={
                "command": "npx",
                "args": ["-y", "@notionhq/notion-mcp-server"],
                "env": {
                    "OPENAPI_MCP_HEADERS": (
                        "{\"Authorization\": \"Bearer " + notion_key + "\", "
                        "\"Notion-Version\": \"2022-06-28\"}"
                    )
                },
            }
        ) as server:
            tool_list = await server.list_tools()
            print(" Outils MCP disponibles :", tool_list)

    except Exception as e:
        print(" Erreur lors de la connexion au serveur MCP :", e)
        traceback.print_exc()

#  Lancer le tout
await main()


 Clé API détectée : ntn_479001...
 Outils MCP disponibles : [Tool(name='API-get-user', title=None, description='Retrieve a user\nError Responses:\n400: 400', inputSchema={'$defs': {}, 'type': 'object', 'properties': {'user_id': {'type': 'string', 'format': 'uuid'}}, 'required': ['user_id']}, outputSchema=None, annotations=None, meta=None), Tool(name='API-get-users', title=None, description='List all users\nError Responses:\n400: 400', inputSchema={'$defs': {}, 'type': 'object', 'properties': {'start_cursor': {'type': 'string', 'description': 'If supplied, this endpoint will return a page of results starting after the cursor provided. If not supplied, this endpoint will return the first page of results.'}, 'page_size': {'type': 'integer', 'default': 100, 'description': 'The number of items from the full list desired in the response. Maximum: 100'}}, 'required': []}, outputSchema=None, annotations=None, meta=None), Tool(name='API-get-self', title=None, description="Retrieve your token's 

### create run() with agent

In [17]:
with open('context/example-2/instructions.md', 'r', encoding='utf-8') as file:
    instructions = file.read()

In [18]:
instructions

'# 🧠 Rôle de l’agent IA\n\nTu es un agent IA spécialisé dans la rédaction de posts LinkedIn à partir des notes enregistrées dans la base Notion intitulée **“Liste de lecture”**.\n\nTu rédiges des posts en **français**, dans un style clair, direct et structuré, sans clichés, sans langage motivationnel, et sans métaphores. Chaque post généré doit être **ajouté automatiquement** dans la base “Liste de lecture” avec les bonnes propriétés.\n\n---\n\n## ✅ Ta mission\n\n1. Lire les entrées disponibles dans la base Notion **“Liste de lecture”**\n2. Générer un **post LinkedIn conforme aux règles ci-dessous**\n3. Ajouter ce post comme **nouvelle ligne dans la base Notion** avec les propriétés suivantes :\n\n| Champ Notion            | Contenu généré ou fixe                     |\n| ----------------------- | ------------------------------------------ |\n| **Nom du contenu**      | Le titre du post LinkedIn                  |\n| **Statut**              | `Idée` ou `En cours d’écriture`            

In [19]:
async def run(mcp_server: MCPServer):
    agent = Agent(
        name="Notion Agent",
        model="gpt-4o",
        instructions=instructions,
        mcp_servers=[mcp_server],
    )
    ModelSettings.tool_choice = "required"
    
    input_items = []

    print("=== Notion AI Agent ===")
    print("Type 'exit' to end the conversation")

    while True:
        # Get user input
        user_input = input("\nUser: ").strip()
        input_items.append({"content": user_input, "role": "user"})
        
        # Check for exit command
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("\nGoodbye!")
            break
            
        if not user_input:
            continue

        print("\nAgent: ", end="", flush=True)
        result = Runner.run_streamed(
            agent,
            input=input_items,
        )
        
        async for event in result.stream_events():
            # We'll ignore the raw responses event deltas for text
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                print(event.data.delta, end="", flush=True)
            elif event.type == "run_item_stream_event":
                if event.item.type == "tool_call_item":
                    print(f"\n-- Calling Tool: {event.item.raw_item.name}...")
                elif event.item.type == "tool_call_output_item":
                    print("-- Tool call completed.")
                elif event.item.type == "message_output_item":
                    input_items.append({"content": f"{event.item.raw_item.content[0].text}", "role": "assistant"})
                else:
                    pass  # Ignore other event types

        print("\n")  # Add a newline after each response

In [20]:
import asyncio
import nest_asyncio
import os
import json
from dotenv import load_dotenv
from agents import Agent, Runner, ModelSettings
from agents.mcp.server import MCPServerStdio
from openai.types.responses import ResponseTextDeltaEvent

# 📌 Appliquer nest_asyncio pour Jupyter
nest_asyncio.apply()

# ✅ Préparer les headers pour l’API MCP Notion
headers = {
    "Authorization": f"Bearer {notion_key}",
    "Notion-Version": "2022-06-28"
}
# 🔁 Fonction principale pour démarrer l'agent
async def run_notion_agent_notebook():
    async with MCPServerStdio(
        params={
            "command": "npx",
            "args": ["-y", "@notionhq/notion-mcp-server"],
            "env": {
                "OPENAPI_MCP_HEADERS": json.dumps(headers)
            },
        }
    ) as mcp_server:

        agent = Agent(
            name="Notion Agent",
            model="gpt-4.1-2025-04-14",
            instructions=instructions,
            mcp_servers=[mcp_server],
        )

        ModelSettings.tool_choice = "required"
        input_items = []

        print("=== Agent IA Notion ===")
        print("Tapez 'exit', 'quit' ou 'bye' pour quitter.\n")

        while True:
            user_input = input("\nVous : ").strip()

            if user_input.lower() in ["exit", "quit", "bye"]:
                print("👋 Session terminée.")
                break

            if not user_input:
                continue

            input_items.append({"content": user_input, "role": "user"})
            print("\nAgent : ", end="", flush=True)

            result = Runner.run_streamed(agent, input_items)

            async for event in result.stream_events():
                if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                    print(event.data.delta, end="", flush=True)
                elif event.type == "run_item_stream_event":
                    if event.item.type == "tool_call_item":
                        print(f"\n-- 🔧 Appel outil : {event.item.raw_item.name}")
                    elif event.item.type == "tool_call_output_item":
                        print("\n-- ✅ Outil terminé.")
                    elif event.item.type == "message_output_item":
                        input_items.append({
                            "content": event.item.raw_item.content[0].text,
                            "role": "assistant"
                        })

# ▶️ Exécute ceci dans une cellule suivante pour démarrer :
await run_notion_agent_notebook()


=== Agent IA Notion ===
Tapez 'exit', 'quit' ou 'bye' pour quitter.




Vous :  donne moi la liste des livres que j'ai lu



Agent : 
-- 🔧 Appel outil : API-post-search

-- ✅ Outil terminé.

-- 🔧 Appel outil : API-post-database-query

-- ✅ Outil terminé.
Je ne peux pas accéder à la liste des livres que vous avez lus car la structure de la base Notion “Liste de lecture” présente un problème technique (“schéma mal formé”). 

Je vous recommande de vérifier ou corriger la structure de la base dans Notion (types de champs, propriété principale, colonnes obligatoires), puis de me reposer la question pour que je puisse extraire la liste demandée. 

Si besoin, dites-moi si vous souhaitez une aide pour structurer correctement cette base.


Vous :  donne moi la liste des livres dans liste de lecture



Agent : 
-- 🔧 Appel outil : API-post-search

-- ✅ Outil terminé.

-- 🔧 Appel outil : API-post-database-query

-- ✅ Outil terminé.
Je ne peux pas extraire la liste des livres de la base “Liste de lecture” car le schéma de la base Notion présente un problème technique (“schema malformé”). Cela bloque toute interrogation automatique.

Voici comment résoudre ce problème :
- Ouvrez la base “Liste de lecture” dans Notion.
- Vérifiez qu’il existe bien une colonne de titre (de type “title”) et d’autres colonnes standard.
- Vérifiez que la structure (types de champs, noms, propriétés obligatoires) est cohérente et non corrompue.
- Partagez bien la base avec l’intégration Notion utilisée.

Vous pourrez ensuite me demander à nouveau la liste des livres, ou toute autre extraction automatique. Si besoin, dites-moi si vous souhaitez des conseils pour structurer ou réparer cette base Notion.

...


Agent : 
-- 🔧 Appel outil : API-post-page

-- ✅ Outil terminé.
Je ne peux pas ajouter le post tant que je n’ai pas l’identifiant unique (UUID) de ta base Notion “Création de contenu”.

Pour créer une nouvelle ligne automatiquement, il me faut ce code unique que tu trouves dans l’URL de ta base Notion, juste après notion.so/… et avant le point d’interrogation ou le slash : par exemple, 123abc45-678d-90ef-gh12-ijkl345678mn. 

Dès que tu fournis cet identifiant (aucune autre information n’est nécessaire), le post sera publié directement dans ta base “Création de contenu”.


Vous :  met dans n'importe quel page et envoie moi le lien



Agent : 
-- 🔧 Appel outil : API-post-search

-- ✅ Outil terminé.

-- 🔧 Appel outil : API-post-page

-- ✅ Outil terminé.
Je n’ai pas pu ajouter le post automatiquement car la structure des contenus n’était pas acceptée par Notion (erreur de format json sur les blocs enfants).

Voici le texte prêt à être copié-collé manuellement dans ta base Notion “Création de contenu” ou “Liste de lectures” :

---

**Structurer sa veille avec une liste de lecture variée**

Gérer sa veille ne se limite plus aux livres : ma “liste de lecture” regroupe aussi podcasts, vidéos et séries.

Centraliser différents formats dans un tableau unique permet de suivre l’avancée, croiser les idées et revenir facilement sur un outil, une méthode, ou une recommandation importante. Voici ce que j’ai récemment ajouté :
- “The road less stupid” (Keith Cunnin : livre)
- “The 12 Week Year” (Brian P. MORAN : livre)
- “Who Not How” (Dan Sullivan : livre)
- “40 Harsh Truths I Know at 40 but Wish I Knew at 20” (Mark Manson : vi


Vous :  bye


👋 Session terminée.


### run agent

In [29]:
await main()
# find me posts about AI consulting

 Clé API détectée : ntn_479001...
 Outils MCP disponibles : [Tool(name='API-get-user', title=None, description='Retrieve a user\nError Responses:\n400: 400', inputSchema={'$defs': {}, 'type': 'object', 'properties': {'user_id': {'type': 'string', 'format': 'uuid'}}, 'required': ['user_id']}, outputSchema=None, annotations=None, meta=None), Tool(name='API-get-users', title=None, description='List all users\nError Responses:\n400: 400', inputSchema={'$defs': {}, 'type': 'object', 'properties': {'start_cursor': {'type': 'string', 'description': 'If supplied, this endpoint will return a page of results starting after the cursor provided. If not supplied, this endpoint will return the first page of results.'}, 'page_size': {'type': 'integer', 'default': 100, 'description': 'The number of items from the full list desired in the response. Maximum: 100'}}, 'required': []}, outputSchema=None, annotations=None, meta=None), Tool(name='API-get-self', title=None, description="Retrieve your token's 


User:  Return post ideas about business and entrepreneurship


In [11]:
# # to run in a .py script use
# if __name__ == "__main__":
#     asyncio.run(main())